In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import numpy as np
import pandas as pd
import sys

from sklearn.datasets import load_diabetes
sys.path.append("..")
from interpretDistill.fourierDistill import FTDistill, FTDistillCV
from interpretDistill.binaryTransformer import BinaryTransformer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from interpretDistill.FTutils import *

In [2]:
X, y = load_diabetes(as_frame = True, return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

In [3]:
bt = BinaryTransformer(depth=1)
X_train = bt.fit_and_transform(X_train, y_train)
X_val = bt.transform(X_val)
# X_val = bt.fit_and_transform(X_val, y_val)
# X_train = bt.transform(X_train)
X_test = bt.transform(X_test)

In [4]:
X_train.shape, X_val.shape, X_test.shape

((176, 19), (177, 19), (89, 19))

In [5]:
rf = RandomForestRegressor(max_depth=4, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=42)

In [6]:
r2_score(y_train, rf.predict(X_train)), r2_score(y_val, rf.predict(X_val)), r2_score(y_test, rf.predict(X_test))

(0.6356421794435572, 0.4445730859243583, 0.3884084201696776)

In [7]:
np.mean((y_train- rf.predict(X_train))**2), np.mean((y_val- rf.predict(X_val))**2), np.mean((y_test- rf.predict(X_test))**2)

(2178.3499294606427, 3426.1063048428523, 3240.3049980560527)

In [8]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd = FTDistill(selection = 'L1L2', lam1=5, lam2=5, size_interactions = 6)
ftd.fit(X_val, rf.predict(X_val), bt.no_interaction)

In [9]:
np.mean((y_train- ftd.predict(X_train))**2), np.mean((y_val- ftd.predict(X_val))**2), np.mean((y_test- ftd.predict(X_test))**2)

(16486.1053869566, 6019.713109659431, 17003.10125482728)

In [10]:
r2_score(y_train, ftd.predict(X_train)), r2_score(y_val, ftd.predict(X_val)), r2_score(y_test, ftd.predict(X_test))

(-1.7575190500924793, 0.02410772503097125, -2.209251463887927)

In [35]:
#ftd = FTDistillCV(selection = 'L1L2', lam1_range=np.arange(0.01, 1.1, 0.33), lam2_range=np.arange(0.01, 1.1, 0.33), size_interactions = 3)
ftd1 = FTDistill(selection = 'L1L2', lam1=10, lam2=10, size_interactions = 6)
ftd1.fit(X_train, rf.predict(X_train))

In [36]:
np.sum(ftd1.regression_model.coef_ != 0)

1420

In [37]:
np.mean((y_train- ftd1.predict(X_train))**2), np.mean((y_val- ftd1.predict(X_val))**2), np.mean((y_test- ftd1.predict(X_test))**2)

(3647.9721520504795, 4699.320278331731, 3934.4003559162925)

In [38]:
r2_score(y_train, ftd1.predict(X_train)), r2_score(y_val, ftd1.predict(X_val)), r2_score(y_test, ftd1.predict(X_test))

(0.3898284362876403, 0.2381646311565334, 0.25740134623024713)